In [13]:
import pandas as pd
import os

file_types = ['trn', 'dev', 'tst']
for f in file_types:
    src_data_df = pd.read_csv(f'../data/SBIC.v2.agg.{f}.csv').sample(frac=0.2)
    result_df = pd.DataFrame()

    result_df['prompt'] = src_data_df['post']
    result_df['completion'] = src_data_df['offensiveYN'].apply(lambda x: 1 if x > 0.5 else 0)

    result_df.to_csv(f'../data/SBIC.v2.agg.{f}.tiny.promted.csv', index=False)
    print(f'openai tools fine_tunes.prepare_data -f data/SBIC.v2.agg.{f}.tiny.promted.csv')

openai tools fine_tunes.prepare_data -f data/SBIC.v2.agg.trn.tiny.promted.csv
openai tools fine_tunes.prepare_data -f data/SBIC.v2.agg.dev.tiny.promted.csv
openai tools fine_tunes.prepare_data -f data/SBIC.v2.agg.tst.tiny.promted.csv


In [10]:
!openai tools fine_tunes.prepare_data -f ../data/SBIC.v2.agg.trn.promted.csv

^C


In [ ]:
!openai api fine_tunes.create -t "data/SBIC.v2.agg.trn.promted_prepared.jsonl" -v "data/SBIC.v2.agg.dev.promted_prepared.jsonl"  --compute_classification_metrics --classification_positive_class " 1" -m davinci

In [6]:
!openai api fine_tunes.results -i ft-HGqkTToaXFMc4RUPlhTyYlbH > result.csv

Error: No results file available for fine-tune ft-HGqkTToaXFMc4RUPlhTyYlbH
